# Rodando os modelos com Cross Validation

In [44]:
# Bibliotecas de manipualção e visualização de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
from sklearn.preprocessing import StandardScaler

# Classes dos modelo
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Funções de avaliação dos modelos
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score, plot_confusion_matrix, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [45]:
df_hepatite = pd.read_csv('HCV-Egy-Data.csv')

In [46]:
# dataset sem outliers
df_hepatite = pd.read_csv('HCV-Egy-Data-no-outlier.csv')

A remoção de outliers mostrou-se eficiente na performace do modelo aumentando em serca de 1% a acurácia.

In [47]:
df_hepatite.head()

,Unnamed: 0,Age,Gender,BMI,Fever,Nausea/Vomting,Headache,Diarrhea,FGba,Jaundice,...,ALT 36,ALT 48,ALT after 24 w,RNA Base,RNA 4,RNA 12,RNA EOT,RNA EF,BhG,Baselinehistological staging
0,1,46,1,29,1,2,2,1,2,2,...,57,123,44,40620,538635,637056,336804,31085,4,2
1,3,49,2,33,1,2,1,2,1,2,...,48,77,33,1041941,449939,585688,744463,582301,10,3
2,5,58,2,22,2,2,2,1,2,2,...,73,114,29,1157452,1086852,5,5,5,4,4
3,6,42,2,26,1,1,2,2,2,2,...,84,80,28,325694,1034008,275095,214566,635157,12,4
4,7,48,2,30,1,1,2,2,1,1,...,96,53,39,641129,72050,787295,370605,506296,12,3


### Seperação da váriável target do dataset

In [48]:
X = df_hepatite.drop(['BhG', 'Baselinehistological staging'], axis=1)
y = df_hepatite['Baselinehistological staging']

### Normalização dos dados

Com a normalização dos dados o modelo melhorou de predição almentou em um valor de 6% na sua acurácia.

In [49]:
scaler = StandardScaler()
scaler.fit(X)

X_train = scaler.transform(X)

In [50]:
#função que retorna um dicionário com os valores dos resultados
def model_results(model, X_train, y_train, X_test, y_test,results_dict_aux):
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    #medindo e armazenando acurácia, f1-score e auc-score no dicionário
    accuracy = model.score(X_test, y_test)
    f1 = f1_score(y_test, y_pred, average='weighted')
    AUC = roc_auc_score(y_test, model.predict_proba(X_test), average='weighted', multi_class='ovo')
    CM = confusion_matrix(y_test, y_pred)

    results_dict_aux['accuracy'].append(accuracy)
    results_dict_aux['f1'].append(f1)
    results_dict_aux['auc'].append(AUC)
    results_dict_aux['cm'].append(CM)

    return results_dict_aux

## Grid Search

Para cada modelo é implementada uma função do grid search. Para ser aplicada em cada um dos 10 conjuntos de treino do 10-fold.

**KNN**

In [51]:
def kNN_grid_search(X_train, y_train):
    #lista com números impares para o número de vizinhos do knn
    k_range = [impar for impar in range(1,32) if (impar%2)!=0]
    #listas com formas de considerar a ditância do vizinho
    weights = ['uniform', 'distance']
    #lista com formas de calcular as distâncias
    dist = ['euclidian','manhattan','chebyshev']
    #dicionário com parêmetros para o gridsearch
    param_grid = {
        'n_neighbors': k_range,
        'weights': weights,
        'metric': dist
    }
    # defining parameter range
    grid = GridSearchCV(knn(), param_grid, cv=3, scoring='accuracy')
    # fitting the model for grid search
    grid.fit(X_train, y_train)

    #utilizando melhores parâmetros calculados pelo gridsearch
    k = grid.best_params_['n_neighbors']
    w = grid.best_params_['weights']
    m = grid.best_params_['metric']

    '''
    print(f"k:{k}, w:{w}, m:{m}\n")
    print(grid.best_score_*100)
    '''

    return (k,w,m)

**Decision Tree**

In [52]:
def dt_grid_search(X_train, y_train):
    #lista com números impares para o número de vizinhos do knn
    max_depth_range = [x for x in range(1,32)]
    #listas com formas de medir a qualidade do 'split'
    criterion_list = ['gini', 'entropy']
    #dicionário com parêmetros para o gridsearch
    param_grid = {
        'max_depth': max_depth_range,
        'criterion': criterion_list,
    }
    # defining parameter range
    grid = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=3, scoring='accuracy')
    # fitting the model for grid search
    grid.fit(X_train, y_train)

    #utilizando melhores parâmetros calculados pelo gridsearch
    md = grid.best_params_['max_depth']
    c = grid.best_params_['criterion']

    '''
    print(f"md:{md}, c:{c}\n")
    print(grid.best_score_*100)    
    '''

    return (md,c)

**MLP**

In [53]:
def mlp_grid_search(X_train, y_train):
    # lista com o número de camadas
    hidden_layer_sizes_list = [(100,), (50, 15, 5), (100, 25, 10)]
    # listas das taxas de aprendizado inicial
    learning_rate_init_list = [0.05, 0.0001]
    # lista de estratégias
    solver_list = ['sgd', 'adam']
    # lista das funções de ativações
    activation_list = ['tanh', 'relu']
    # lista da forma da taxa de aprendizado
    learning_rate_lsit = ['constant', 'adaptive']
    # dicionário com parêmetros para o gridsearch
    param_grid = {
        'hidden_layer_sizes': hidden_layer_sizes_list,
        'activation': activation_list,
        'solver': solver_list,
        'learning_rate': learning_rate_lsit,
        'learning_rate_init': learning_rate_init_list,
    }
    # defining parameter range
    grid = GridSearchCV(MLPClassifier(), param_grid, cv=3, scoring='accuracy')
    # fitting the model for grid search
    grid.fit(X_train, y_train)

    # utilizando melhores parâmetros calculados pelo gridsearch
    hls = grid.best_params_['hidden_layer_sizes']
    a = grid.best_params_['activation']
    s = grid.best_params_['solver']
    lr = grid.best_params_['learning_rate']
    lri = grid.best_params_['learning_rate_init']

    '''
    print(f"hls:{hls}, a:{a}, s:{s}, lr:{lr}, lri:{lri}\n")
    print(grid.best_score_*100)
    '''

    return (hls, a, s, lr, lri)

**SVM**

In [54]:
def svm_grid_search(X_train, y_train):
    #lista com números impares para o número de vizinhos do knn
    C_list = [1, 10, 100]
    #listas com formas de medir a qualidade do 'split'
    kernel_list = ['linear', 'poly'] #['linear', 'poly', 'rbf', 'sigmoid']
    #dicionário com parêmetros para o gridsearch
    param_grid = {
        'C': C_list,
        'kernel': kernel_list,
    }
    # defining parameter range
    grid = GridSearchCV(SVC(), param_grid, cv=3, scoring='accuracy')
    # fitting the model for grid search
    grid.fit(X_train, y_train)

    #utilizando melhores parâmetros calculados pelo gridsearch
    c = grid.best_params_['C']
    k = grid.best_params_['kernel']

    '''
    print(f"C:{c}, k:{k}\n")
    print(grid.best_score_*100)
    '''

    return (c,k)

## 10-Fold

Com esse método são criados 10 datasets de treino e 10 datasets de test com uma divisão de 90% para treino e 10% para teste em cada divisão.

O conjunto de treino sera divido mais uma vez em treino e validação (isso é feito dendo da função `GridSearchCV` para que então seja aplicado o GridSearch e assim obtenha-se os melhores parâmetros. Por fim, tendo os melhores parâmetros, utiliza-se o conjunto de teste para que se possa avaliar os resultados.

Esses resultados são obtidos de cada fold e então se tira a média deles para obter-se a avaliação final de cada modelo.

In [55]:
#função que roda os modelos em cada uma das divisões do 10-fold
#e imprime a média e o desvio padrão dos resultados

def evaluate_model_with_kfold(kf):
    results_dict_models = {}
    # listas e dicionarios para salvar as métricas dos resultados de todas as interacoes
    accuracies_list = []
    f1_score_list = []
    auc_list = []
    confusion_matrix_list = []
    
    results_dict_KNN = {
        'accuracy': accuracies_list,
        'f1': f1_score_list,
        'auc': auc_list,
        'cm': confusion_matrix_list
    }
    results_dict_DT = {
        'accuracy': accuracies_list,
        'f1': f1_score_list,
        'auc': auc_list,
        'cm': confusion_matrix_list
    }
    results_dict_MLP = {
        'accuracy': accuracies_list,
        'f1': f1_score_list,
        'auc': auc_list,
        'cm': confusion_matrix_list
    }
    results_dict_SVM = {
        'accuracy': accuracies_list,
        'f1': f1_score_list,
        'auc': auc_list,
        'cm': confusion_matrix_list
    }
    
    count = 1
    
    for train, test in kf.split(X, y):
        X_train, y_train, X_test, y_test = X.iloc[train], y.iloc[train], X.iloc[test], y.iloc[test]        
        
        #kNN
        k,w,m = kNN_grid_search(X_train, y_train)
        model = knn(n_neighbors=k, weights=w, metric=m)
        results_dict_KNN = model_results(model, X_train, y_train, X_test, y_test, results_dict_KNN)
        if (count == 10):
            results_dict_models['knn'] = results_dict_KNN

        '''
        #DT
        md,c = dt_grid_search(X_train, y_train)
        model = DecisionTreeClassifier(max_depth=md, criterion=c, random_state=199)
        results_dict_models['DT'] = model_results(model, X_train, y_train, X_test, y_test)


        #MLP 
        hls, a, s, lr, lri = mlp_grid_search(X_train,y_train)
        model = MLPClassifier(
            hidden_layer_sizes=hls, 
            activation=a, 
            solver=s, 
            learning_rate=lr, 
            learning_rate_init=lri, 
            max_iter=2000, 
            tol=0.000001,
            random_state=199
        )
        results_dict_models['MLP'] = model_results(model, X_train, y_train, X_test, y_test)
        results_dict_models['MLP']['best_param'] = [hls, a, s, lr, lri]

        #GNB 
        model = GaussianNB()
        results_dict_models['GNB'] = model_results(model, X_train, y_train, X_test, y_test)

        #SVM
        c, k = svm_grid_search(X_train, y_train)
        model = SVC(C=c, kernel=k, probability=True, random_state=199)
        results_dict_models['SVM'] = model_results(model, X_train, y_train, X_test, y_test)

        '''
        
        count = count + 1

    # a cada interação calcula a média e o desvio padrão da 
    # acurácia, f1-score, auc-scor e matriz de confusão de cada modelo
    for model_key in results_dict_models.keys():
        accuracies = np.array(results_dict_models[model_key]['accuracy'])
        f1 = np.array(results_dict_models[model_key]['f1'])
        auc = np.array(results_dict_models[model_key]['auc'])
        conf_matrix = np.array(results_dict_models[model_key]['cm'])

        print(model_key)
        print("Acurácia média (desvio): %.6f +- (%.9f)" %(accuracies.mean(), accuracies.std()))
        print("F1-score média (desvio): %.6f +- (%.9f)" %(f1.mean(), f1.std()))
        print("AUC média (desvio): %.6f +- (%.9f)\n" %(auc.mean(), auc.std()))
        print(f"Matriz de Confusão:  \n{sum(conf_matrix)*0.1}\n")

In [56]:
#%%time
evaluate_model_with_kfold(StratifiedKFold(n_splits=10, shuffle=True, random_state=199))

C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\neighbors\_classification.py", line 179, in fit
    return self._fit(X, y)
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\neighbors\_base.py", line 399, in _fit
    self._check_algorithm_metric()
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\neighbors\_base.py", line 342, in _check_algorithm_metric
    raise ValueError("Metric '%s' not valid. Use "
ValueError: Metric 'euclidian' not valid. Use sorted(sklearn.ne

C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan 0.25040258 0.25040258 0.23913043 0.25120773
 0.25523349 0.24476651 0.24476651 0.25362319 0.24074074 0.24235105
 0.24879227 0.2431562  0.25120773 0.25040258 0.25845411 0.2431562
 0.25201288 0.24798712 0.25603865 0.25040258 0.25362319 0.2568438
 0.24959742 0.25362319 0.24557166 0.24557166 0.26086957 0.26086957
 0.2568438  0.26247987 0.2568438  0.25925926 0.25523349 0.25523349
 0.25603865 0.25523349 0.26086957 0.26086957 0.25201288 0.25442834
 0.25201288 0.24396135 0.24718196 0.239

C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\neighbors\_classification.py", line 179, in fit
    return self._fit(X, y)
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\neighbors\_base.py", line 399, in _fit
    self._check_algorithm_metric()
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\neighbors\_base.py", line 342, in _check_algorithm_metric
    raise ValueError("Metric '%s' not valid. Use "
ValueError: Metric 'euclidian' not valid. Use sorted(sklearn.ne

C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\neighbors\_classification.py", line 179, in fit
    return self._fit(X, y)
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\neighbors\_base.py", line 399, in _fit
    self._check_algorithm_metric()
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\neighbors\_base.py", line 342, in _check_algorithm_metric
    raise ValueError("Metric '%s' not valid. Use "
ValueError: Metric 'euclidian' not valid. Use sorted(sklearn.ne

C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan 0.24778922 0.24778922 0.23652678 0.25019886
 0.23973381 0.24858856 0.23974546 0.24296219 0.24216673 0.23813127
 0.2477931  0.24618668 0.24859632 0.2437751  0.25584463 0.24538347
 0.24535242 0.23894224 0.24537765 0.24858856 0.25181111 0.24939953
 0.24215315 0.24457443 0.24457249 0.23814485 0.24939565 0.24779116
 0.24456667 0.24055061 0.24778146 0.24054479 0.24538153 0.24538153
 0.23410744 0.24779699 0.24777952 0.24778728 0.23412103 0.24376928
 0.23893254 0.24699377 0.2502008  0.2

...

C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan 0.24619832 0.24619832 0.25261044 0.25906137
 0.26066585 0.25423627 0.26468968 0.25986652 0.26228004 0.25906137
 0.26309877 0.27033739 0.26227034 0.25825621 0.25744912 0.25666143
 0.25824651 0.25906137 0.25503754 0.25665367 0.24537377 0.25744718
 0.24938789 0.25181499 0.24939759 0.25020662 0.25904585 0.25101178
 0.25986458 0.25180335 0.26307355 0.25180335 0.25503948 0.25503948
 0.25502784 0.25826009 0.26791805 0.26148071 0.25504918 0.25665949
 0.24942087 0.25504918 0.2647052  0.2

C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\neighbors\_classification.py", line 179, in fit
    return self._fit(X, y)
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\neighbors\_base.py", line 399, in _fit
    self._check_algorithm_metric()
  File "C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\neighbors\_base.py", line 342, in _check_algorithm_metric
    raise ValueError("Metric '%s' not valid. Use "
ValueError: Metric 'euclidian' not valid. Use sorted(sklearn.ne

C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan 0.24940535 0.24940535 0.25340201 0.25421687
 0.26466445 0.26547543 0.26870186 0.2751392  0.26709543 0.26387288
 0.26870186 0.26307355 0.26548125 0.26307937 0.27192635 0.27353084
 0.26952253 0.26066003 0.26710708 0.26228004 0.25342142 0.26146712
 0.26871932 0.2687135  0.26710708 0.26307743 0.27031023 0.26468386
 0.26789089 0.27030441 0.25905167 0.26789671 0.25422851 0.25422851
 0.2662864  0.2638884  0.26386706 0.25743554 0.26067361 0.26145936
 0.26308519 0.26389228 0.26709738 0.2

knn
Acurácia média (desvio): 0.257069 +- (0.043435224)
F1-score média (desvio): 0.255121 +- (0.043835213)
AUC média (desvio): 0.504385 +- (0.030905041)

Matriz de Confusão:  
[[ 8.3  7.5  9.3  8.4]
 [ 8.7  6.7  8.9  8.8]
 [ 9.7  7.2 10.2  8.4]
 [ 8.3  7.6  9.8 10.3]]



C:\Users\jwsl\anaconda3\envs\DataScience\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan 0.24860408 0.24860408 0.25583105 0.25020856
 0.25341171 0.24859438 0.2437654  0.24698601 0.25181693 0.2502008
 0.261471   0.25262208 0.25664203 0.25423821 0.26148265 0.25504724
 0.25745688 0.25262984 0.2526182  0.257453   0.26146712 0.2526376
 0.25583881 0.25423433 0.25904779 0.2461828  0.25823875 0.26468386
 0.25742778 0.25663427 0.25340589 0.25261238 0.24458219 0.24458219
 0.25100984 0.2510176  0.25020662 0.25985876 0.25584463 0.25502978
 0.25422851 0.24617892 0.25182469 0.253